In [25]:
import bpy
import bmesh
import socket
import struct
import threading


vx_type = "3fi"
cr_type = "2f2f4fi"
fc_type = "qiii"


def list_unpack(buf, element):
    sz = struct.calcsize(element)
    l = len(buf) // sz
    return [struct.unpack(element, buf[x * sz:(x+1) * sz]) for x in range(l)]

def read_int(stream):
    l, *_ = struct.unpack("i", stream.recv(4))
    print(f"Int: {l}")
    return l

def read_array(stream, element, max_buffer_size=4096):
    element_size = struct.calcsize(element)
    step = max_buffer_size // element_size
    l, *_ = struct.unpack("i", stream.recv(4))
    to_recv = l * element_size
    print(f"len {l}")
    data = b''
    i = 0
    while True:
        frag = stream.recv(min(max_buffer_size, to_recv))
        print(f"frag {len(frag)}")
        if len(frag) == 0:
            break
        data += frag
        i += len(frag) // element_size
        to_recv -= len(frag)
        if i >= l:
            break
    return data

def write_array(stream, element, values, max_buffer_size=4096):
    element_size = struct.calcsize(element)
    step = max_buffer_size // element_size
    stream.sendall(struct.pack("i", len(values)))
    buf = b''.join((struct.pack(element, *x) for x in values))
    totalsent = 0
    while totalsent < len(buf):
        sent = stream.send(buf[totalsent:])
        if sent == 0:
            raise RuntimeError("socket connection broken")
        totalsent = totalsent + sent

def read_mesh(stream):
    vx = list_unpack(read_array(stream, vx_type), vx_type)
    cr = list_unpack(read_array(stream, cr_type), cr_type)
    fc = list_unpack(read_array(stream, fc_type), fc_type)
    return (vx, cr, fc)

def write_mesh(stream, mesh):
    (vx, cr, fc) = mesh
    write_array(stream, vx_type, vx)
    write_array(stream, cr_type, cr)
    write_array(stream, fc_type, fc)


class commands:
    Brushify = 4

def read_header(stream):
    code, *_ = struct.unpack("i", stream.recv(4))
    print(f"code {code}")
    if code != 0:
        raise ValueError()

def write_header(stream, code):
    stream.sendall(struct.pack("i", code))

def make_bmesh(mesh):
    (verts, corners, faces) = mesh
    m = bmesh.new()
    vflags = m.verts.layers.int.new("vflags")
    for x in verts:
        v = m.verts.new(x[0:3])
        v[vflags] = x[3]
    m.verts.ensure_lookup_table()
    uv = m.loops.layers.uv.new('uv')
    lm = m.loops.layers.uv.new('lm')
    col =  m.loops.layers.float_color.new('color')
    flags = m.faces.layers.int.new("flags")
    flags_hi = m.faces.layers.int.new("flags_hi")
    for x in faces:
        corns = corners[x[1]:x[1]+x[2]]
        faceverts = [m.verts[x[8]] for x in corns]
        f = m.faces.new(faceverts)
        f.material_index = x[3]
        f[flags] = x[0] & 0xFFFFFFFF
        f[flags_hi] = (x[0] >> 32) & 0xFFFFFFFF
        for i, c in enumerate(corns):
            f.loops[i][uv].uv = c[0:2]
            f.loops[i][lm].uv = c[2:4]
            f.loops[i][col] = c[4:8]
    m.faces.ensure_lookup_table()
    return m

def object_to_mesh(obj):
    if obj.type != "MESH": raise TypeError(f"Invalid object type: {obj.type}")
    m = bmesh.new()
    m.from_mesh(obj.data)
    m.verts.ensure_lookup_table()
    m.faces.ensure_lookup_table()
    vflags = m.verts.layers.int.get("vflags")
    uv = m.loops.layers.uv.get('uv')
    lm = m.loops.layers.uv.get('lm')
    col =  m.loops.layers.float_color.get('color')
    flags = m.faces.layers.int.get("flags")
    flags_hi = m.faces.layers.int.get("flags_hi")
    vx = []
    for x in m.verts:
        vx.append((*x.co, x[vflags] if vflags is not None else 0))
    cr = []
    fc = []
    for f in m.faces:
        n = len(cr)
        nn = len(f.verts)
        for i, v in enumerate(f.verts):
            cr.append((
                *(f.loops[i][uv].uv if uv is not None else (0.0, 0.0)),
                *(f.loops[i][lm].uv if lm is not None else (0.0, 0.0)),
                *(f.loops[i][col] if col is not None else (0.0, 0.0, 0.0, 0.0)),
                v.index
            ))
        fhi = f[flags_hi] if flags_hi is not None else 0
        flo = f[flags] if flags is not None else 0
        fc.append(((fhi << 32) | flo, n, nn, f.material_index))
    return (vx, cr, fc)

def create_collection(name):
    col = bpy.data.collections.new(name)
    bpy.context.scene.collection.children.link(col)
    return col

def add_to_collection(col, name, mesh):
    m = bpy.data.meshes.new(name)
    mesh.to_mesh(m)
    o = bpy.data.objects.new(name, m)
    col.objects.link(o)
    return o

def request(code, send, process):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect(("127.0.0.1", 2232))
        write_header(s, code)
        send(s)
        return process(s)

def request_brushify(obj):
    col = create_collection(f"{obj.name}_brushes")
    m = object_to_mesh(obj)
    def process(stream):
        read_header(stream)
        num = read_int(stream)
        for i in range(num):
            m = read_mesh(stream)
            add_to_collection(col, f"brush-{i}", make_bmesh(m))
    request(commands.Brushify, lambda s: write_mesh(s, m), process)


def listen_socket(port, collection, cancellation_token):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("127.0.0.1", port))
        s.listen()
        while cancellation_token[0]:
            conn, addr = s.accept()
            with conn:
                read_header(conn)
                data = read_mesh(conn)
                print(len(data))
                print(len(data[0]), len(data[1]), len(data[2]))
                add_to_collection(collection, "mesh", make_bmesh(data))

def listen_for_polys(port, collection):
    cancellation_token = [True]
    t = threading.Thread(target=listen_socket, args=(port, collection, cancellation_token))
    t.start()
    return t, cancellation_token

class BSP_OP_BrushifyFaces(bpy.types.Operator):
    bl_idname = "bsp.brushify_faces"
    bl_label = "Brushify"
    bl_description = "Brushify."
    bl_options = {'REGISTER', 'INTERNAL'}
    @classmethod
    def poll(cls, context):
        return context.object is not None and context.object.type == "MESH"
    def execute(self, context):
        request_brushify(context.object)
        return {'FINISHED'}

class BSP_PT_ScenePanel(bpy.types.Panel):
    bl_space_type = 'PROPERTIES'
    bl_region_type = 'WINDOW'
    bl_context = "object"
    bl_label = "BSP"
    def draw(self, context):
        layout = self.layout
        layout.operator(BSP_OP_BrushifyFaces.bl_idname, icon='MOD_EXPLODE', text='')

classes = [
    BSP_OP_BrushifyFaces,
    BSP_PT_ScenePanel,
]
props = []

def register():
    for cls in classes:
        bpy.utils.register_class(cls)

def unregister():
    for c in classes:
        clst = c.__bases__[0].bl_rna_get_subclass_py(c.__name__)
        if clst:
            bpy.utils.unregister_class(clst)

if __name__ == '__main__':
    unregister()
    register()

In [38]:
request_brushify(bpy.data.objects["Cube"])

In [39]:
ss = _

In [ ]:
def request_brushify(mesh):
    

In [3]:
c = create_collection("polys_debug")

In [8]:
t, canc = listen_for_polys(22332, c)

Exception in thread Thread-5 (listen_socket):
Traceback (most recent call last):
  File "/home/rantrave/miniconda3/envs/bpy/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/rantrave/miniconda3/envs/bpy/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_7081/2944978834.py", line 14, in listen_socket
  File "/tmp/ipykernel_7081/1237257472.py", line 27, in make_bmesh
TypeError: expected BMLoopUV, not a tuple


code 0
len 8
frag 128
len 24
frag 864
len 6
frag 144
3
8 24 7
[(-12.345678329467773, -12.359590530395508, -12.348592758178711, 0), (-12.345678329467773, -12.359590530395508, 12.345678329467773, 0), (-12.345678329467773, 12.345678329467773, 12.345678329467773, 0), (-12.345678329467773, 12.345678329467773, -12.348592758178711, 0), (37.05094909667969, -12.359590530395508, -12.348592758178711, 0), (37.05094909667969, 12.345678329467773, -12.348592758178711, 0), (37.05094909667969, 12.345678329467773, 12.345678329467773, 0), (37.05094909667969, -12.359590530395508, 12.345678329467773, 0)]
[(0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1), (0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 7), (0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 6), (0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0), (0.

False

In [12]:
c.all_objects

bpy.data.collections['polys_debug'].all_objects

In [14]:
t.is_alive()

False

In [30]:
tt = threading.enumerate()[3]

True

In [3]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("127.0.0.1", 2232))
    s.sendall(struct.pack("i", 3))
    data = get_mesh(s)

ConnectionRefusedError: [Errno 111] Connection refused